In [1]:
import tensorflow as tf
from models._12_net import detect_12Net
import numpy as np
from PIL import Image
from utils.utilities import batch_selection
from utils.sampling import GetSample
from utils import save
import numpy as np
from utils.params import *
import sys
from utils.cnn_utils import *
from utils.utilities import *
from PIL import ImageDraw

In [2]:
pos_db_12 = save.loader('pos_db_12.txt')
neg_db_12 = save.loader('neg_db_12.txt')

Loaded from D:\programing project\python project\trying stuffpickle/
Loaded from D:\programing project\python project\trying stuffpickle/


In [3]:
g = tf.Graph()
with g.as_default():
    input_12_node = tf.placeholder(tf.float32, [None,12,12,3])
    target_12_node = tf.placeholder(tf.float32, [None,1])
    keep_prob = tf.placeholder(tf.float32)
    learning_rate = tf.placeholder(tf.float32)
    net_12 = detect_12Net(input_12_node, target_12_node, keep_prob, learning_rate)

In [4]:
sess = tf.InteractiveSession(config=tf.ConfigProto(
  allow_soft_placement=True, log_device_placement=True), graph=g)
sess.run(tf.global_variables_initializer())
# train_writer = tf.summary.FileWriter('./logs/train/Det_12', sess.graph)

In [5]:
test_dir = 'Test_source/test/'
test_face = 'Test_source/test_face/'
test_img = ['41009506_p1.jpg', '32819641_p0.jpg', '2015-02-08-709570.jpg', '46255545_p0.jpg', '42435403_p0.jpg']


face_list = [face_crop_scale(test_face + path) for path in test_img]

In [6]:
lr = [5e-2, 1e-2, 5e-3, 1e-3, 5e-4]
bias_init = [0.00]
pepochs = 500
neg_db_12 = neg_db_12[:10080*2,:]
#neg_db_12 = neg_db_12[:10080,:]

In [8]:
#total_iteration = p_epochs * int(neg_db_12.shape[0] / p_neg_batch)
bat_len = int(neg_db_12.shape[0] / p_neg_batch)
bad_cnt = 0
single_loss = 0
max_epo = 0
suggested_number = 200.0


for b in bias_init:
    for l in lr:
        record_l = l
        total_iteration = pepochs * int(neg_db_12.shape[0] / p_neg_batch)
        print('**Bias_init:{}   Learning_rate:{}   epo:{}'.format(b,l,pepochs))
        batch_sel_pos = batch_selection()
        batch_sel_neg = batch_selection()
        cur_score_list = [-0.1 for _ in range(5)]
        pre_det = [0 for _ in range(5)]
        max_score_list = [0.0 for _ in range(5)]
        suggest_amount = [0 for _ in range(5)]
        max_ratio = [0 for _ in range(5)]
        max_print = [0 for _ in range(5)]
        max_de = np.asarray([0 for _ in range(5)])
        max_sug = np.asarray([0 for _ in range(5)])
        max_avg = 0.0
        avg_avg = 0.0
        ch_ratio = np.zeros(5)
        bad_det = 0
        inputs = np.zeros((p_batch_size, 12, 12, 3))
        targets = np.zeros((p_batch_size, 1))

        g = tf.Graph()
        with g.as_default():
            input_12_node = tf.placeholder(tf.float32, [None,None,None,3])
            target_12_node = tf.placeholder(tf.float32, [None,1])
            keep_prob = tf.placeholder(tf.float32)
            learning_rate = tf.placeholder(tf.float32)
            net_12 = detect_12Net(input_12_node, target_12_node, keep_prob, learning_rate)

        sess = tf.InteractiveSession(config=tf.ConfigProto(
          allow_soft_placement=True, log_device_placement=True), graph=g)
        sess.run(tf.global_variables_initializer())

        #train_writer = tf.summary.FileWriter('./logs/train/Det_12/lr={},b={},ep={},GS'.format(lr,b,pepochs), sess.graph)
        for epo in range(pepochs):
            batch_sel_pos = batch_selection()
            batch_sel_neg = batch_selection()
            
            loss = 0
            if (epo + 1) % 5 == 0:
                
                #pre_det = cur_det
                cur_det = [detection_windows(img_scalar(test_dir + t_img), 
                                            p_net_12, net_12, input_12_node, 
                                            keep_prob, thres=.8) for t_img in test_img]
                suggest_amount = np.asarray([len(result_box) for result_box in cur_det])
                cur_score_list = np.asarray([len(det_box_eval(face_list[i], cur_det[i])) 
                                  if det_box_eval(face_list[i], cur_det[i]).any() else 0 for i in range(len(test_img))])
                list_ratio = [cur_score_list[i] / suggest_amount[i] for i in range(5)]
                avg_avg = np.average(cur_score_list / suggest_amount)
                #ch_ratio = np.array(cur_det) / (np.array(pre_det) + 1e-9)
                if max_avg <= avg_avg:
                    max_avg = avg_avg
                    max_sug = suggest_amount
                    max_ratio = list_ratio
                    max_de = cur_score_list
                    bad_det = 0
                    saver = tf.train.Saver()
                    saver.save(sess, p_model_dir + 'det_12/max/12_net_Adam_,lr={}b={}Trying.ckpt'.format(record_l,b))
                    max_epo = epo
                else:
                    bad_det += 1
                
            if (bad_det+1) % 5 == 0:
                l = l * .98
            if bad_det >= 100:
                print(' Breaking\n')
                break
            
            for i in range(bat_len):
                iter_index = epo*int(neg_db_12.shape[0] / p_neg_batch) + i
                inputs[:p_pos_batch,] = batch_sel_pos.next_batch_single(pos_db_12, p_pos_batch)
                inputs[p_pos_batch:,] = batch_sel_neg.next_batch_single(neg_db_12, p_neg_batch)
                targets[:p_pos_batch,] = np.ones((p_pos_batch,1))
                targets[p_pos_batch:,] = np.zeros((p_neg_batch,1))

                feed_regular = {
                    input_12_node: inputs,
                    target_12_node: targets,
                    keep_prob:1.0,
                    learning_rate:l
                }
                feed_train = {
                    input_12_node: inputs,
                    target_12_node: targets,
                    keep_prob:.5,
                    learning_rate:l
                }

                loss += net_12.loss.eval(feed_regular)
                single_loss = net_12.loss.eval(feed_regular)

                if single_loss > 0.1:
                    bad_cnt += 1
                
                net_12.train_step.run(feed_train)

                """if iter_index % 100 == 0:
                    summary = net_12.merged.eval(feed_regular)
                    train_writer.add_summary(summary, iter_index)"""

                sys.stdout.write('\rEpoch:' + str(epo+1) + \
                                 '     loss/single_loss:' + str(loss / (i+1))[:6] + '/' + str(single_loss)[:10] + \
                                 '     iteration:' + str(iter_index) + '/' + str(total_iteration) + \
                                 '     Bad_CNT:' + str(bad_cnt) + \
                                 '     Bad_ratio: ' + str(100*float(bad_cnt)/((iter_index%(10 * bat_len))+1))[:10] + "%" + \
                                 '                      Right Detect Ratio: ' + str(cur_score_list) + \
                                 '     Suggested_total: ' + str(suggest_amount) + \
                                 '     Max_detect: ' + str(max_de) + '        Now_ratio: ' + str(avg_avg) + \
                                 '     Max_ratio: ' + str(max_avg) + '       Max_suggested: ' + str(max_sug) + \
                                 '     Bad_det: ' + str(bad_det) + '    L:' + str(l) + '   Max_epo: ' + str(max_epo))

        print('\n')
        # '     Avg_score:' + str(np.sum(max_score_list)/len(test_img)) + \
        # return xmin, ymin, xmax, ymax, confidence score, img_crop obj, scale
        
        
        #for t_img in test_img:
            #detec_wins = detection_windows(img_scalar(test_dir + t_img), p_net_12, net_12, input_12_node, keep_prob, thres=0.1)
            #list_det.append(len(detec_wins))
        #print('Detection numbers: \n', list_det)
        #saver = tf.train.Saver()
        #saver.save(sess, p_model_dir + 'det_12/12_net_AdaDelta_l={},b={},iter={}.ckpt'.format(l,b,iter_index))

**Bias_init:0.0   Learning_rate:0.05   epo:500
Epoch:500     loss/single_loss:0.0731/0.0843756     iteration:34999/35000     Bad_CNT:8903     Bad_ratio: 1271.85714%                      Right Detect Ratio: [0 2 1 5 1]     Suggested_total: [31  5 17 13 14]     Max_detect: [0 3 2 3 0]        Now_ratio: 0.182973497091     Max_ratio: 0.253333333333       Max_suggested: [18  5 12  6  5]     Bad_det: 77    L:0.008115528676075822   Max_epo: 114

**Bias_init:0.0   Learning_rate:0.01   epo:500
Epoch:110     loss/single_loss:0.0310/0.0201437     iteration:7683/35000     Bad_CNT:9080     Bad_ratio: 1327.48538%                      Right Detect Ratio: [ 9  8  4 10 12]     Suggested_total: [359 108  84 152 117]     Max_detect: [ 9  8  4 10 12]        Now_ratio: 0.0630232671649     Max_ratio: 0.0630232671649       Max_suggested: [359 108  84 152 117]     Bad_det: 0    L:0.006676079717550942   Max_epo: 109

KeyboardInterrupt: 

In [30]:
suggest_amount, cur_score_list

(array([2349, 2367,  814, 2462, 1882]), array([21, 15, 11, 20, 23]))

In [32]:
np.average(cur_score_list / suggest_amount)

0.0098270283757586948

In [33]:
np.average(max_de / max_sug)

0.011026046349365892

In [29]:
max_sug, max_de

(array([2801, 1901,  945, 2205, 1645]), array([25, 17, 13, 21, 23]))

In [21]:
np.sum([19, 17, 16, 19, 18]) / np.sum([4062, 1853, 2504, 1656, 1286]) 

0.0078338174456473904

In [17]:
["{0:.5f}".format(l) for l in lll]

['0.00963', '0.01064', '0.01248', '0.00982', '0.01421']

In [2]:
Get = GetSample()
pos_db_12 = Get.get_pos_img(12)
save.saver('pos_db_12.txt', pos_db_12)

Samples Checked..
IMG_INDEX:112/112
Finished
Saved in D:\programing project\python project\trying stuffpickle/


In [2]:
Get = GetSample()
pos_db_12 = save.loader('pos_db_12.txt')

Loaded from D:\programing project\python project\trying stuffpickle/


In [10]:
pos_db_12.shape

(672, 12, 12, 3)

In [5]:
batch_sel_pos = batch_selection()
batch_sel_neg = batch_selection()

In [6]:
inputs = np.zeros((p_batch_size, 12, 12, 3))
targets = np.zeros((p_batch_size, 1))

In [7]:
g = tf.Graph()
with g.as_default():
    input_12_node = tf.placeholder(tf.float32, [None,12,12,3])
    target_12_node = tf.placeholder(tf.float32, [None,1])
    keep_prob = tf.placeholder(tf.float32)
    learning_rate = tf.placeholder(tf.float32)
    net_12 = detect_12Net(input_12_node, target_12_node, keep_prob, learning_rate)

In [ ]:
sess = tf.InteractiveSession(config=tf.ConfigProto(
  allow_soft_placement=True, log_device_placement=True), graph=g)
sess.run(tf.global_variables_initializer())
# train_writer = tf.summary.FileWriter('./logs/train/Det_12', sess.graph)

In [4]:
neg_db_12 = Get.get_neg_img(12)
total_iteration = p_epochs * int(neg_db_12.shape[0] / p_neg_batch)
bat_len = int(neg_db_12.shape[0] / p_neg_batch)

In [11]:
test_dir = 'Test_source/test/'
test_face = 'Test_source/test_face/'
test_img = ['41009506_p1.jpg', '32819641_p0.jpg', '2015-02-08-709570.jpg', '46255545_p0.jpg', '42435403_p0.jpg']


face_list = [face_crop_scale(test_face + path) for path in test_img]

In [12]:
lr = [1e-1, 8e-2, 5e-2, 1e-2, 8e-3]
bias_init = [0.00]
pepochs = 800

In [6]:
lr = [1e-4]
bias_init = [0.00]
pepochs = 800

In [13]:
neg_db_12 = Get.get_neg_img(12)
#total_iteration = p_epochs * int(neg_db_12.shape[0] / p_neg_batch)
bat_len = int(neg_db_12.shape[0] / p_neg_batch)
bad_cnt = 0
single_loss = 0
max_epo = 0
suggested_number = 200.0

for l in lr:
    for b in bias_init:
        record_l = l
        total_iteration = pepochs * int(neg_db_12.shape[0] / p_neg_batch)
        print('**Bias_init:{}   Learning_rate:{}   epo:{}'.format(b,l,pepochs))
        batch_sel_pos = batch_selection()
        batch_sel_neg = batch_selection()
        cur_score_list = [-0.1 for _ in range(5)]
        pre_det = [0 for _ in range(5)]
        max_score_list = [0.0 for _ in range(5)]
        ch_ratio = np.zeros(5)
        bad_det = 0
        inputs = np.zeros((p_batch_size, 12, 12, 3))
        targets = np.zeros((p_batch_size, 1))

        g = tf.Graph()
        with g.as_default():
            input_12_node = tf.placeholder(tf.float32, [None,None,None,3])
            target_12_node = tf.placeholder(tf.float32, [None,1])
            keep_prob = tf.placeholder(tf.float32)
            learning_rate = tf.placeholder(tf.float32)
            net_12 = detect_12Net(input_12_node, target_12_node, keep_prob, learning_rate)

        sess = tf.InteractiveSession(config=tf.ConfigProto(
          allow_soft_placement=True, log_device_placement=True), graph=g)
        sess.run(tf.global_variables_initializer())

        #train_writer = tf.summary.FileWriter('./logs/train/Det_12/lr={},b={},ep={},GS'.format(lr,b,pepochs), sess.graph)
        for epo in range(pepochs):
            batch_sel_pos = batch_selection()
            batch_sel_neg = batch_selection()
            #if (epo + 1) == 200:
                #l = 6e-2
            # Recreate the neg_sample every 10th epoch
            """if (np.sum(np.asarray(min_det)<100) >= 1) and (l>5e-4):
                l = 5e-3"""
                
            if (epo + 1) % 10 == 0:
                #if bad_cnt < (0.1 * bat_len * 10):
                neg_db_12 = Get.get_neg_img(12)
                bad_cnt = 0
                #else: 
                #bad_cnt = 0
            loss = 0
            if (epo + 1) % 10 == 0:
                
                #pre_det = cur_det
                cur_det = [detection_windows_ver2(img_scalar(test_dir + t_img), 
                                            p_net_12, net_12, input_12_node, 
                                            keep_prob) for t_img in test_img]
                
                cur_score_list = [len(det_box_eval(face_list[i], cur_det[i])) / suggested_number 
                                  if det_box_eval(face_list[i], cur_det[i]).any() else 0 for i in range(len(test_img))] 
                #ch_ratio = np.array(cur_det) / (np.array(pre_det) + 1e-9)
                if np.sum(max_score_list) < np.sum(cur_score_list):
                    max_score_list = cur_score_list
                    bad_det = 0
                    saver = tf.train.Saver()
                    saver.save(sess, p_model_dir + 'det_12/max/12_net_AdaDelta_,lr={}b={}Trying.ckpt'.format(record_l,b))
                    max_epo = epo
                else:
                    bad_det += 1
                #if (np.sum(ch_ratio>.98)) >= 2:
                    #bad_det += 1
                #else:
                    #bad_det = 0
                
            if (bad_det+1) % 10 == 0:
                l = l * .92
            if bad_det >= 24:
                print(' Breaking\n')
                break
            
            for i in range(bat_len):
                iter_index = epo*int(neg_db_12.shape[0] / p_neg_batch) + i
                inputs[:p_pos_batch,] = batch_sel_pos.next_batch_single(pos_db_12, p_pos_batch)
                inputs[p_pos_batch:,] = batch_sel_neg.next_batch_single(neg_db_12, p_neg_batch)
                targets[:p_pos_batch,] = np.ones((p_pos_batch,1))
                targets[p_pos_batch:,] = np.zeros((p_neg_batch,1))

                feed_regular = {
                    input_12_node: inputs,
                    target_12_node: targets,
                    keep_prob:1.0,
                    learning_rate:l
                }
                feed_train = {
                    input_12_node: inputs,
                    target_12_node: targets,
                    keep_prob:.65,
                    learning_rate:l
                }

                loss += net_12.loss.eval(feed_regular)
                single_loss = net_12.loss.eval(feed_regular)

                if single_loss > 0.1:
                    bad_cnt += 1
                
                net_12.train_step.run(feed_train)

                """if iter_index % 100 == 0:
                    summary = net_12.merged.eval(feed_regular)
                    train_writer.add_summary(summary, iter_index)"""

                sys.stdout.write('\rEpoch:' + str(epo+1) + \
                                 '     loss/single_loss:' + str(loss / (i+1))[:6] + '/' + str(single_loss)[:10] + \
                                 '     iteration:' + str(iter_index) + '/' + str(total_iteration) + \
                                 '     Bad_CNT:' + str(bad_cnt) + \
                                 '     Bad_ratio: ' + str(100*float(bad_cnt)/((iter_index%(10 * bat_len))+1))[:10] + "%" + \
                                 '                      Right Detect Ratio: ' + str(cur_score_list) + \
                                 '     Max_Detect: ' + str(max_score_list) + \
                                 '     Avg_ratio: ' + str(np.sum(max_score_list)/len(test_img)) + \
                                 '     Bad_det: ' + str(bad_det) + '    L:' + str(l) + '   Max_epo: ' + str(max_epo))

        print('\n')
        # '     Avg_score:' + str(np.sum(max_score_list)/len(test_img)) + \
        # return xmin, ymin, xmax, ymax, confidence score, img_crop obj, scale
        
        
        #for t_img in test_img:
            #detec_wins = detection_windows(img_scalar(test_dir + t_img), p_net_12, net_12, input_12_node, keep_prob, thres=0.1)
            #list_det.append(len(detec_wins))
        #print('Detection numbers: \n', list_det)
        #saver = tf.train.Saver()
        #saver.save(sess, p_model_dir + 'det_12/12_net_AdaDelta_l={},b={},iter={}.ckpt'.format(l,b,iter_index))

**Bias_init:0.0   Learning_rate:0.1   epo:800
Epoch:569     loss/single_loss:0.0815/0.0817171     iteration:6827/9600     Bad_CNT:2     Bad_ratio: 1.85185185%                      Right Detect Ratio: [0.025, 0.06, 0.05, 0.045, 0.065]     Max_Detect: [0.025, 0.06, 0.065, 0.04, 0.065]     Avg_ratio: 0.051     Bad_det: 23    L:0.018869332916279676   Max_epo: 329 Breaking



**Bias_init:0.0   Learning_rate:0.08   epo:800
Epoch:499     loss/single_loss:0.1297/0.129896     iteration:5987/9600     Bad_CNT:120     Bad_ratio: 111.111111%                      Right Detect Ratio: [0, 0.06, 0.035, 0.035, 0.05]     Max_Detect: [0, 0.065, 0.04, 0.03, 0.05]     Avg_ratio: 0.037     Bad_det: 23    L:0.006557296286187064   Max_epo: 259 Breaking



**Bias_init:0.0   Learning_rate:0.05   epo:800
Epoch:509     loss/single_loss:0.1093/0.0979574     iteration:6107/9600     Bad_CNT:103     Bad_ratio: 95.3703703%                      Right Detect Ratio: [0.02, 0.065, 0.04, 0.04, 0.06]     Max_Detect: [0.015, 

In [ ]:
tf.train.RMSPropOptimizer

In [ ]:
list1 = [1509, 111, 192, 2087, 906]

In [13]:
list_arr = np.asarray(list1)

In [14]:
np.sum(list_arr<200)

2

In [6]:
test_dir = 'Test_source/test/'
test_img = '41009506_p1.png'

In [7]:
# return xmin, ymin, xmax, ymax, confidence score, img_crop obj, scale
detec_wins = detection_windows(img_scalar(test_dir + test_img), p_net_12, net_12, input_12_node, keep_prob, thres=0.1)

In [8]:
len(detec_wins)

12843

In [12]:
draw = img_scalar(test_dir + test_img)
for win in detec_wins:
    ImageDraw.Draw(draw).rectangle(list(np.asarray(win[:4],dtype=np.int32)), outline='red')